In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import transforms as t
import pathlib
import pandas as pd
import tensorflow_datasets as tfds
import glob
import re    
from tensorflow.keras import layers

In [13]:
files = np.sort(glob.glob("./data/*"))
label_classes = ["focus", "neutral"]
batch_size = 32

In [14]:
def initial_process(files, label_classes):
    for file in files:
        data = []
        labels = []
        temp = pd.read_csv(open(file, "r"))
        data.append(temp)
        for j in range(len(label_classes)):
            if label_classes[j] in file:
                labels.append(j)
        yield data, labels

In [15]:
data = initial_process
(files, label_classes)
dataset = tf.data.Dataset.from_generator(initial_process, args=[files, label_classes], output_signature=(tf.TensorSpec(shape=(1, 51, 1), dtype=tf.float32), tf.TensorSpec(shape=(1,), dtype=tf.int32)))
dataset = dataset.repeat(1000)

In [16]:
train_ds = dataset.take(int(0.8 * len(list(dataset))))
val_ds = dataset.skip(int(0.8 * len(list(dataset))))

In [17]:
def squeeze_data(data, label):
    data = tf.squeeze(data)
    tf.reshape(data, [51])
    return data, tf.squeeze(label)

In [18]:
'''from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(1,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])

model.summary()
'''

"from tensorflow.keras import layers\n\nmodel = tf.keras.Sequential([\n    layers.Dense(32, activation='relu', input_shape=(1,)),\n    layers.Dense(64, activation='relu'),\n    layers.Dense(1, activation='linear')\n])\n\nmodel.summary()\n"

In [19]:
model = tf.keras.Sequential([
    layers.Conv1D(32, 3, activation="relu", input_shape=(51,1)),
    layers.MaxPool1D(2),
    layers.Conv1D(64, 3, activation="relu"),
    layers.MaxPool1D(2),
    layers.Flatten(),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(2, activation="softmax")
])
model.summary()

C:\Users\Streamer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 49, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 24, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 22, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 704)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │        22,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,458 (115.07 KB)

 Trainable params: 29,458 (115.07 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
model.fit(train_ds, validation_data=val_ds, epochs=10, batch_size=batch_size)

Epoch 1/10
   7980/Unknown 13s 1ms/step - accuracy: 0.9771 - loss: 0.0683

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8000/8000 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9772 - loss: 0.0681 - val_accuracy: 0.9995 - val_loss: 3.8128e-07
Epoch 2/10
 103/8000 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 1.0000 - loss: 3.9846e-07

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


7976/8000 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 1.7127e-07

...

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


7992/8000 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0000e+00

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8000/8000 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.9995 - val_loss: 0.0000e+00


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [22]:
model.save("model.keras")